<p><font size="6" color='grey'> <b>
Machine Learning
</b></font> </br></p>
<p><font size="5" color='grey'> <b>
Supervised Learning - Decision Tree - Titanic
</b></font> </br></p>

---


# 0  | Install & Import
***

In [1]:
# Install
!uv pip install --system -q dtreeviz

In [19]:
# Import
from pandas import read_excel, DataFrame

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_text, export_graphviz
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    classification_report,
)

import plotly.express as px
import plotly.subplots as sp

import graphviz
import dtreeviz

In [16]:
# Warnung ausstellen
import warnings
warnings.filterwarnings("ignore")

# 1  | Understand
***

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Aufgabe verstehen</br>
✅ Daten sammeln</br>
✅ Statistische Analyse (Min, Max, Mean, Korrelation, ...)</br>
✅ Datenvisualisierung (Streudiagramm, Box-Plot, ...)</br>
✅ Prepare Schritte festlegen</br>

<p><font color='black' size="5">
📒 Anwendungsfall
</font></p>

Dies ist der legendäre Titanic ML-Wettbewerb – die beste erste Herausforderung, um in ML-Modellierung einzutauchen.

Die Aufgabe ist einfach: Verwenden Sie maschinelles Lernen, um ein Modell zu erstellen, das vorhersagt, welche Passagiere den Schiffbruch der Titanic überlebt haben.

[Titanic Org](https://www.encyclopedia-titanica.org/)

[DataSet](https://www.openml.org/search?type=data&status=active&id=41265)

[Info](https://www.kaggle.com/competitions/titanic/data)



**Datenfelder:**   
+ Age: Alter
+ Fare: Ticketpreis
+ Sex: Geschlecht (0 = männlich, 1 = weiblich)
+ sibsp: Der Datensatz definiert Familienbeziehungen auf diese Weise ... Geschwister = Bruder, Schwester, Stiefbruder, Stiefschwester Ehepartner = Ehemann, Ehefrau (Geliebte und Verlobte wurden ignoriert)
+ parch: Der Datensatz definiert Familienbeziehungen auf diese Weise ... Elternteil = Mutter, Vater Kind = Tochter, Sohn, Stieftochter, Stiefsohn. Einige Kinder reisten nur mit einem Kindermädchen, daher ist für sie Parch=0
+ Pclass: Passagierklasse, 1.- 3. Klasse
+ Embarked: Hafen der Einschiffung

In [21]:
df = read_excel(
    "https://raw.githubusercontent.com/ralf-42/ML_Intro/main/02_daten/05_tabellen/titanic.xlsx",
    usecols=["pclass", "survived", "sex", "age", "sibsp", "parch"],
)

In [22]:
data = df.copy()
target = data.pop("survived")

<p><font color='black' size="5">
🔎 EDA (Exploratory Data Analysis) mit Pandas
</font></p>

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   pclass  1309 non-null   int64  
 1   sex     1309 non-null   object 
 2   age     1309 non-null   object 
 3   sibsp   1046 non-null   float64
 4   parch   1309 non-null   int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 51.3+ KB


In [24]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
pclass,1309.0,2.294882,0.837836,1.0,2.0,3.0,3.0,3.0
sibsp,1046.0,95.855641,654.480326,1.0,21.0,29.0,41.0,9167.0
parch,1309.0,0.498854,1.041658,0.0,0.0,0.0,1.0,8.0


In [25]:
data.groupby("sex").count()

,pclass,age,sibsp,parch
sex,,,,
Capt. Edward Gifford,1,1,1,1
Col. Archibald IV,1,1,1,1
Col. John,1,1,1,1
Col. John Jacob,1,1,1,1
Col. Oberst Alfons,1,1,1,1
...,...,...,...,...
Rev. Robert James,1,1,1,1
Rev. Thomas Roussel Davids,1,1,1,1
Rev. William,1,1,1,1


In [26]:
target.value_counts()

,count
survived,
0,809
1,500


In [27]:
data[data.age.isnull()].describe().T

,count,mean,std,min,25%,50%,75%,max
pclass,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sibsp,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
parch,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 2 | Prepare

---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Nicht benötigte Features löschen</br>
✅ Datentyp ermitteln/ändern</br>
✅ Duplikate ermitteln/löschen</br>
✅ Missing Values behandeln</br>
✅ Ausreißer behandeln</br>
✅ Kategorischer Features Kodieren</br>
✅ Numerischer Features skalieren</br>
✅ Feature-Engineering (neue Features schaffen)</br>
✅ Dimensionalität reduzieren</br>
✅ Resampling (Over-/Undersampling)</br>
✅ Pipeline erstellen/konfigurieren</br>
✅ Train-Test-Split durchführen</br>


<p><font color='black' size="5">
🔢 Kodierung
</font></p>

In [30]:
data.sex.replace({"male": 0, "female": 1}, inplace=True)

,pclass,sex,age,sibsp,parch
0,1,Miss. Elisabeth Walton,female,29.0,0
1,1,Master. Hudson Trevor,male,9167.0,1
2,1,Miss. Helen Loraine,female,2.0,1
3,1,Mr. Hudson Joshua Creighton,male,30.0,1
4,1,Mrs. Hudson J C (Bessie Waldo Daniels),female,25.0,1
...,...,...,...,...,...
1304,3,Miss. Hileni,female,145.0,1
1305,3,Miss. Thamine,female,NaN,1
1306,3,Mr. Mapriededer,male,265.0,0
1307,3,Mr. Ortin,male,27.0,0


<p><font color='black' size="5">
 ☠️Missing Values
</font></p>

In [29]:
data.age.fillna(data.age.mean(), inplace=True)

TypeError: Could not convert string 'femalemalefemalemalefemalemalefemalemalefemalemalemalefemalefemalefemalemalemalemalefemalefemalemalemalefemalemalefemalefemalemalemalefemalefemalemalemalemalefemalefemalemalefemalefemalemalemalemalemalefemalefemalefemalefemalemalemalemalefemalemalefemalemalemalemalemalefemalemalefemalemalefemalemalefemalemalefemalemalefemalefemalefemalemalefemalemalemalefemalefemalemalemalefemalemalefemalefemalemalemalefemalefemalemalefemalemalemalefemalemalefemalemalefemalemalemalefemalemalefemalefemalefemalemalemalefemalefemalefemalefemalemalefemalefemalemalemalefemalefemalefemalemalemalefemalefemalemalemalemalefemalefemalemalefemalemalemalefemalemalefemalefemalefemalemalemalefemalemalemalefemalemalefemalemalefemalemalemalefemalemalefemalemalemalefemalemalemalemalefemalemalefemalemalemalemalefemalefemalefemalemalefemalemalemalemalefemalefemalefemalemalemalemalemalemalemalefemalemalefemalemalefemalefemalefemalemalemalemalefemalefemalefemalemalefemalemalefemalefemal

<p><font color='black' size="5">
✂️ Train-Test-Split
</font></p>


In [ ]:
data_train, data_test, target_train, target_test = train_test_split(
    data, target, test_size=0.20, random_state=42, stratify=target
)

In [ ]:
data_train.shape, data_test.shape, target_train.shape, target_test.shape

# 3 | Modeling
---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Modellauswahl treffen</br>
✅ Pipeline erweitern/konfigurieren</br>
✅ Training durchführen</br>
✅ Hyperparameter Tuning</br>
✅ Cross-Valdiation</br>
✅ Bootstrapping</br>
✅ Regularization</br>

<p><font color='black' size="5">
🏃 Modellauswahl & Training
</font></p>

In [ ]:
model = DecisionTreeClassifier(max_depth=3, random_state=42)

In [ ]:
model.fit(data_train, target_train)

# 4 | Evaluate
---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Prognose (Train, Test) erstellen</br>
✅ Modellgüte prüfen</br>
✅ Residuenanalyse erstellen</br>
✅ Feature Importance/Selektion prüfen</br>
✅ Robustheitstest erstellen</br>
✅ Modellinterpretation erstellen</br>
✅ Sensitivitätsanalyse erstellen</br>
✅ Kommunikation (Key Takeaways)</br>

<p><font color='black' size="5">
🔭 Prognose
</font></p>

In [ ]:
target_train_pred = model.predict(data_train)
target_test_pred = model.predict(data_test)

<p><font color='black' size="5">
📱 Confusion Matrix
</font></p>

In [ ]:
conf_matrix = confusion_matrix(target_test, target_test_pred)
display_labels_ = ["Not Survived", "Survived"]
disp = ConfusionMatrixDisplay(conf_matrix, display_labels=display_labels_)
disp.plot(cmap="Blues")

In [ ]:
print(classification_report(target_test, target_test_pred, target_names=display_labels_))

<p><font color='black' size="5">
🎯 Accuracy
</font></p>

In [ ]:
acc_train = accuracy_score(target_train, target_train_pred) * 100
print(f"Modell: {model} -- Train -- Accuracy: {acc_train:5.2f}")

In [ ]:
acc_test = accuracy_score(target_test, target_test_pred) * 100
print(f"Modell: {model} -- Test -- Accuracy: {acc_test:5.2f}%")

<p><font color='black' size="5">
🎲Aufbau Analysewürfel
</font></p>

In [ ]:
# Übernahme der Testdaten
cube = data_test.copy()
cube.reset_index(inplace=True)

In [ ]:
# Übernahme Target real & predict
cube["real"] = DataFrame(target_test.values, columns=["real"])
cube["predict"] = DataFrame(target_test_pred, columns=["predict"])

<p><font color='black' size="5">
📊 Visualisierung real vs predict
</font></p>

In [ ]:
# Histogramm
title_ = "Histogramm real vs predict"
fig = px.histogram(cube, x=["real", "predict"], nbins=2, title=title_)
fig.update_layout(barmode="group", bargap=0.1, width=600, height=600)
fig.show()

<p><font color='black' size="5">
📛 Fehlerhafte Vorhersagen
</font></p>

In [ ]:
# real <> predict
cube[cube.real != cube.predict].describe().T

In [ ]:
cube[cube.real != cube.predict]

<p><font color='black' size="5">
📝 Einzelne Vorhersage
</font></p>

In [ ]:
# 2 neue Datensätze werden zur Prognose an das Modell übergeben: Rose & Jack (Winslet/diCaprio)
new_data = {
    "pclass": [1, 3],
    "sex": [1, 0],
    "age": [22, 23],
    "sibsp": [0, 0],
    "parch": [0, 0],
}
new = DataFrame(new_data)

In [ ]:
# Vorhersage erstellen Rose & Jack
model.predict(new)

<p><font color='black' size="5">
❗Feature Importance
</font></p>

In [ ]:
title_ = "Feature Importance Titanic"
px.bar(
    x=model.feature_importances_, y=data.columns, title=title_, width=800, height=600
).update_yaxes(categoryorder="total ascending")

<p><font color='black' size="5">
🌳 Darstellen des Entscheidungsbaums
</font></p>

Erstellen einen Textbericht, der die Regeln eines Entscheidungsbaums zeiget.
Für die Anzeige class_name muss ein Update auf sklearn 1.4 erfolgen.

In [ ]:
# tree_rules = export_text(model, feature_names=list(model.feature_names_in_), class_names=display_labels_, spacing=7, decimals=1,  show_weights=True)
tree_rules = export_text(
    model,
    feature_names=list(model.feature_names_in_),
    spacing=7,
    decimals=1,
    show_weights=True,
)
print(tree_rules)

Als Grafik mit graphviz

In [ ]:
# DOT data
dot_data = export_graphviz(
    model,
    out_file=None,
    feature_names=data.columns,
    class_names=display_labels_,
    filled=True,
)

Aufbau Knoten im Entscheidungsbaum

<img src="https://raw.githubusercontent.com/ralf-42/Image/main/knoten.png" class="logo" width="480"/>





In [ ]:
# Draw graph
graph = graphviz.Source(dot_data, format="png")
graph

Als Grafik mit dtreeviz

In [ ]:
viz_model = dtreeviz.model(
    model,
    data_test,
    target_test,
    target_name="survived",
    class_names=display_labels_,
    feature_names=data.columns,
)

In [ ]:
viz_model.view(scale=1.5, fontname="Monospace")

In [ ]:
viz_model.view(scale=1.5, orientation="LR", fontname="Monospace")

In [ ]:
# local Explanation
one = data_test.iloc[13]
viz_model.view(x=one, fontname="Monospace")

In [ ]:
# local Explanation
viz_model.view(x=one, show_just_path=True, fontname="Monospace")

In [ ]:
tree_img = viz_model.view(scale=0.8, fontname="Monospace")
tree_img.save("tree.svg")

# 5 | Deploy
---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Modellexport und -speicherung</br>
✅ Abhängigkeiten und Umgebung</br>
✅ Sicherheit und Datenschutz</br>
✅ In die Produktion integrieren</br>
✅ Tests und Validierung</br>
✅ Dokumentation & Wartung</br>